# 🌳 PointNet for 3D Tree Species Classification

## Method: Direct 3D Point Cloud Classification
This notebook implements the **PointNet** deep learning model for direct 3D point cloud classification.

## Key Features:
1. **Direct 3D processing**: Works directly with point cloud data
2. **Farthest Point Sampling (FPS)**: Samples exactly 1024 points from variable-sized clouds
3. **Data normalization**: Centers and scales points to unit sphere
4. **PointNet architecture**: Complete implementation with T-Net and classification head
5. **7-class classification**: Adapted for tree species classification

## Pipeline:
1. **Data loading**: Load `.xyz`, `.pts` files from train/test folders
2. **Preprocessing**: FPS sampling + normalization
3. **Model training**: PointNet with T-Net transformation
4. **Evaluation**: Classification report and confusion matrix

Expected: Good performance on geometric features of tree species

In [20]:
# PointNet 3D Tree Classification - Complete Pipeline
import os
import time
import torch
import warnings
import numpy as np
import open3d as o3d
import torch.nn as nn
import seaborn as sns
from pathlib import Path
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

warnings.filterwarnings("ignore")

# PointNet Configuration
NUM_POINTS = 1024        # Fixed number of points after FPS
BATCH_SIZE = 32          # Batch size for training
LEARNING_RATE = 1e-3     # Learning rate
NUM_EPOCHS = 100         # Training epochs
PATIENCE = 20            # Early stopping patience
DEVICE = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')

print("🌳 PointNet 3D Tree Species Classification")
print("="*50)
print(f"   Device: {DEVICE}")
print(f"   Points per cloud: {NUM_POINTS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Method: Direct 3D point cloud processing")

🌳 PointNet 3D Tree Species Classification
   Device: mps
   Points per cloud: 1024
   Batch size: 32
   Learning rate: 0.001
   Method: Direct 3D point cloud processing


In [21]:
# PointCloud Dataset Class with FPS and Normalization
class PointCloudDataset(Dataset):
    """Dataset for 3D point clouds with FPS sampling and normalization"""
    
    def __init__(self, data_path, num_points=1024, split='train'):
        self.data_path = Path(data_path)
        self.num_points = num_points
        self.split = split
        
        # Load file paths and labels
        self.file_paths, self.labels = self._load_data()
        print(f"   Loaded {len(self.file_paths)} {split} files")
    
    def _load_data(self):
        """Load all point cloud file paths and labels"""
        file_paths, labels = [], []
        
        for species_dir in self.data_path.iterdir():
            if species_dir.is_dir():
                # Get all point cloud files
                files = list(species_dir.glob("*.xyz")) + list(species_dir.glob("*.pts")) + list(species_dir.glob("*.txt"))
                print(f"   {species_dir.name}: {len(files)} files")
                
                file_paths.extend(files)
                labels.extend([species_dir.name] * len(files))
        
        return file_paths, labels
    
    def _load_point_cloud(self, file_path):
        """Load point cloud from file"""
        try:
            # Try different loading methods based on file extension
            if file_path.suffix.lower() in ['.xyz', '.txt']:
                # Load as text file
                points = np.loadtxt(file_path, usecols=(0, 1, 2))
            elif file_path.suffix.lower() == '.pts':
                # Load .pts file (sometimes has header)
                with open(file_path, 'r') as f:
                    lines = f.readlines()
                
                # Skip header if present
                start_idx = 0
                for i, line in enumerate(lines):
                    try:
                        float(line.split()[0])
                        start_idx = i
                        break
                    except:
                        continue
                
                points = []
                for line in lines[start_idx:]:
                    coords = line.strip().split()
                    if len(coords) >= 3:
                        points.append([float(coords[0]), float(coords[1]), float(coords[2])])
                
                points = np.array(points)
            else:
                # Fallback: try Open3D
                pcd = o3d.io.read_point_cloud(str(file_path))
                points = np.asarray(pcd.points)
            
            return points
            
        except Exception as e:
            print(f"Warning: Could not load {file_path}: {e}")
            # Return a dummy point cloud
            return np.random.randn(100, 3)
    
    def _farthest_point_sampling(self, points, num_samples):
        """Perform Farthest Point Sampling using Open3D"""
        if len(points) <= num_samples:
            # If we have fewer points, pad with random noise
            padding = np.random.randn(num_samples - len(points), 3) * 0.01
            points = np.vstack([points, padding])
            return points
        
        try:
            # Create Open3D point cloud
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(points)
            
            # Perform FPS
            sampled_pcd = pcd.farthest_point_down_sample(num_samples)
            return np.asarray(sampled_pcd.points)
            
        except Exception as e:
            print(f"Warning: FPS failed, using random sampling: {e}")
            # Fallback to random sampling
            indices = np.random.choice(len(points), num_samples, replace=False)
            return points[indices]
    
    def _normalize_point_cloud(self, points):
        """Normalize point cloud to unit sphere"""
        # Center the point cloud
        centroid = np.mean(points, axis=0)
        points = points - centroid
        
        # Scale to unit sphere
        max_dist = np.max(np.sqrt(np.sum(points**2, axis=1)))
        if max_dist > 0:
            points = points / max_dist
        
        return points
    
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        # Load point cloud
        points = self._load_point_cloud(self.file_paths[idx])
        
        # Apply FPS to get exactly num_points
        points = self._farthest_point_sampling(points, self.num_points)
        
        # Normalize to unit sphere
        points = self._normalize_point_cloud(points)
        
        # Convert to tensor (N, 3) -> (3, N) for PointNet
        points = torch.FloatTensor(points).transpose(0, 1)
        
        return points, self.labels[idx]

print("✅ PointCloudDataset class defined with FPS and normalization")

✅ PointCloudDataset class defined with FPS and normalization


In [22]:
# T-Net (Transformation Network) for PointNet
class TNet(nn.Module):
    """T-Net transformation network for spatial transformer"""
    
    def __init__(self, k=3):
        super(TNet, self).__init__()
        self.k = k
        
        # Shared MLPs
        self.conv1 = nn.Conv1d(k, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k * k)
        
        # Batch normalization
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)
        
        # Initialize transformation matrix as identity
        self.register_buffer('identity', torch.eye(k).flatten())
    
    def forward(self, x):
        batch_size = x.size(0)
        
        # Shared MLPs
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        
        # Global max pooling
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(batch_size, -1)
        
        # Fully connected layers
        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)
        
        # Add identity transformation
        x = x + self.identity.expand(batch_size, -1)
        
        # Reshape to transformation matrix
        x = x.view(batch_size, self.k, self.k)
        
        return x

print("✅ T-Net transformation network defined")

✅ T-Net transformation network defined


In [23]:
# PointNet Classification Model
class PointNet(nn.Module):
    """PointNet model for 3D point cloud classification"""
    
    def __init__(self, num_classes=7, num_points=1024):
        super(PointNet, self).__init__()
        self.num_classes = num_classes
        self.num_points = num_points
        
        # Input transformation (T-Net)
        self.input_transform = TNet(k=3)
        
        # Shared MLPs for point-wise features
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 64, 1)
        
        # Feature transformation (T-Net)
        self.feature_transform = TNet(k=64)
        
        # More shared MLPs
        self.conv3 = nn.Conv1d(64, 64, 1)
        self.conv4 = nn.Conv1d(64, 128, 1)
        self.conv5 = nn.Conv1d(128, 1024, 1)
        
        # Batch normalization layers
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(128)
        self.bn5 = nn.BatchNorm1d(1024)
        
        # Classification head
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        
        self.bn6 = nn.BatchNorm1d(512)
        self.bn7 = nn.BatchNorm1d(256)
        
        self.dropout = nn.Dropout(0.3)
        
        print(f"🏗️ PointNet Model:")
        print(f"   Input: {num_points} points × 3 coordinates")
        print(f"   Features: T-Net + Shared MLPs + Global Max Pool")
        print(f"   Output: {num_classes} tree species classes")
        print(f"   Architecture: Input T-Net + Feature T-Net + Classifier")
    
    def forward(self, x):
        batch_size = x.size(0)
        num_points = x.size(2)
        
        # Input transformation
        input_trans = self.input_transform(x)  # (B, 3, 3)
        x = torch.bmm(input_trans, x)  # Apply transformation
        
        # First set of shared MLPs
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        
        # Feature transformation
        feature_trans = self.feature_transform(x)  # (B, 64, 64)
        x = torch.bmm(feature_trans, x)  # Apply feature transformation
        
        # Second set of shared MLPs
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        
        # Global max pooling to get shape signature
        x = torch.max(x, 2, keepdim=True)[0]  # (B, 1024, 1)
        x = x.view(batch_size, -1)  # (B, 1024)
        
        # Classification head
        x = F.relu(self.bn6(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn7(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x

print("✅ PointNet classification model defined")

✅ PointNet classification model defined


In [24]:
# Data Loading and Preprocessing
print("📁 Loading 3D point cloud datasets...")

# Load datasets
train_dataset = PointCloudDataset("../../../train", num_points=NUM_POINTS, split='train')
test_dataset = PointCloudDataset("../../../test", num_points=NUM_POINTS, split='test')

# Get class names and encode labels
all_labels = train_dataset.labels + test_dataset.labels
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

class_names = label_encoder.classes_
num_classes = len(class_names)

print(f"\n📊 Dataset Information:")
print(f"   Classes: {num_classes} tree species")
print(f"   Class names: {list(class_names)}")
print(f"   Train samples: {len(train_dataset)}")
print(f"   Test samples: {len(test_dataset)}")

# Encode labels for datasets
train_dataset.labels = label_encoder.transform(train_dataset.labels)
test_dataset.labels = label_encoder.transform(test_dataset.labels)

# Create validation split from training data
train_indices = list(range(len(train_dataset)))
train_idx, val_idx = train_test_split(
    train_indices, test_size=0.2, random_state=42, 
    stratify=train_dataset.labels
)

# Create data loaders
train_loader = DataLoader(
    torch.utils.data.Subset(train_dataset, train_idx),
    batch_size=BATCH_SIZE, shuffle=True, num_workers=0
)

val_loader = DataLoader(
    torch.utils.data.Subset(train_dataset, val_idx),
    batch_size=BATCH_SIZE, shuffle=False, num_workers=0
)

test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0
)

print(f"\n✅ Data loaders created:")
print(f"   Train batches: {len(train_loader)}")
print(f"   Validation batches: {len(val_loader)}")
print(f"   Test batches: {len(test_loader)}")

# Test data loading
print(f"\n🔍 Testing data loading...")
for batch_idx, (points, labels) in enumerate(train_loader):
    print(f"   Batch {batch_idx + 1}: Points shape: {points.shape}, Labels shape: {labels.shape}")
    print(f"   Points range: [{points.min():.3f}, {points.max():.3f}]")
    if batch_idx == 0:  # Just test first batch
        break

print("✅ Data loading successful!")

📁 Loading 3D point cloud datasets...
   Oak: 18 files
   Douglas Fir: 147 files
   Spruce: 127 files
   Pine: 20 files
   Ash: 32 files
   Red Oak: 81 files
   Beech: 132 files
   Loaded 557 train files
   Oak: 4 files
   Douglas Fir: 36 files
   Spruce: 31 files
   Pine: 5 files
   Ash: 7 files
   Red Oak: 19 files
   Beech: 32 files
   Loaded 134 test files

📊 Dataset Information:
   Classes: 7 tree species
   Class names: [np.str_('Ash'), np.str_('Beech'), np.str_('Douglas Fir'), np.str_('Oak'), np.str_('Pine'), np.str_('Red Oak'), np.str_('Spruce')]
   Train samples: 557
   Test samples: 134

✅ Data loaders created:
   Train batches: 14
   Validation batches: 4
   Test batches: 5

🔍 Testing data loading...
   Batch 1: Points shape: torch.Size([32, 3, 1024]), Labels shape: torch.Size([32])
   Points range: [-1.000, 0.829]
✅ Data loading successful!


In [25]:
# Model Initialization and Training Setup
print("🏗️ Initializing PointNet model...")

# Create model
model = PointNet(num_classes=num_classes, num_points=NUM_POINTS).to(DEVICE)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=10
)

def accuracy_metric(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return torch.sum(preds == labels).item()

print(f"\n🔧 Training Setup:")
print(f"   Loss: CrossEntropyLoss")
print(f"   Optimizer: Adam @ {LEARNING_RATE}")
print(f"   Scheduler: ReduceLROnPlateau")
print(f"   Device: {DEVICE}")
print(f"   Ready for training!")

🏗️ Initializing PointNet model...
🏗️ PointNet Model:
   Input: 1024 points × 3 coordinates
   Features: T-Net + Shared MLPs + Global Max Pool
   Output: 7 tree species classes
   Architecture: Input T-Net + Feature T-Net + Classifier
   Total parameters: 3,471,568
   Trainable parameters: 3,471,568

🔧 Training Setup:
   Loss: CrossEntropyLoss
   Optimizer: Adam @ 0.001
   Scheduler: ReduceLROnPlateau
   Device: mps
   Ready for training!


In [ ]:
# PointNet Training Loop
def train_pointnet(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, patience):
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    
    print("🚀 POINTNET TRAINING (3D Point Clouds)")
    print("="*50)
    
    for epoch in range(num_epochs):
        epoch_start = time.time()
        
        # Training phase
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        
        for batch_idx, (points, labels) in enumerate(train_loader):
            points, labels = points.to(DEVICE), labels.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(points)
            loss = criterion(outputs, labels)
            loss.backward()
            
            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            train_loss += loss.item()
            train_correct += accuracy_metric(outputs, labels)
            train_total += labels.size(0)
            
            # Progress logging
            if batch_idx % 10 == 0:
                batch_acc = accuracy_metric(outputs, labels) / labels.size(0)
                print(f"   Epoch {epoch+1:2d} | Batch {batch_idx:2d}/{len(train_loader)} | Loss: {loss.item():.4f} | Acc: {batch_acc:.3f}")
        
        # Validation phase
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        
        with torch.no_grad():
            for points, labels in val_loader:
                points, labels = points.to(DEVICE), labels.to(DEVICE)
                outputs = model(points)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                val_correct += accuracy_metric(outputs, labels)
                val_total += labels.size(0)
        
        # Calculate metrics
        train_loss /= len(train_loader)
        train_acc = train_correct / train_total
        val_loss /= len(val_loader)
        val_acc = val_correct / val_total
        
        # Update scheduler
        scheduler.step(val_acc)
        
        # Store history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        epoch_time = time.time() - epoch_start
        
        # Print epoch summary
        print(f"   Epoch {epoch+1:2d}: Train={train_acc:.3f}, Val={val_acc:.3f}, "
              f"Loss={train_loss:.4f}, Time={epoch_time:.1f}s")
        
        # Best model tracking
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()
            patience_counter = 0
            print(f"      ⭐ NEW BEST: {best_val_acc:.3f}")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"   ⏹️ Early stopping at epoch {epoch+1}")
                break
        
        print("-" * 50)
    
    # Load best model
    if best_model_state:
        model.load_state_dict(best_model_state)
        print(f"✅ Best model restored: {best_val_acc:.3f}")
    
    return history, best_val_acc

# Start training
print("🌳 Starting PointNet Training...")
start_time = time.time()
history, best_val_acc = train_pointnet(
    model, train_loader, val_loader, criterion, optimizer, scheduler,
    NUM_EPOCHS, PATIENCE
)
training_time = time.time() - start_time

print(f"\n⏱️ Training completed in {training_time:.1f}s")

🌳 Starting PointNet Training...
🚀 POINTNET TRAINING (3D Point Clouds)
   Epoch  1 | Batch  0/14 | Loss: 1.9338 | Acc: 0.281
   Epoch  1 | Batch 10/14 | Loss: 1.6840 | Acc: 0.375
   Epoch  1: Train=0.326, Val=0.223, Loss=1.7338, Time=141.5s
      ⭐ NEW BEST: 0.223
--------------------------------------------------
   Epoch  2 | Batch  0/14 | Loss: 1.3696 | Acc: 0.375
   Epoch  2 | Batch 10/14 | Loss: 1.5162 | Acc: 0.375
   Epoch  2: Train=0.438, Val=0.062, Loss=1.3658, Time=156.6s
--------------------------------------------------
   Epoch  3 | Batch  0/14 | Loss: 1.1403 | Acc: 0.438
   Epoch  3 | Batch 10/14 | Loss: 1.2095 | Acc: 0.531
   Epoch  3: Train=0.488, Val=0.161, Loss=1.2119, Time=153.4s
--------------------------------------------------
   Epoch  4 | Batch  0/14 | Loss: 0.9586 | Acc: 0.531
   Epoch  4 | Batch 10/14 | Loss: 1.0907 | Acc: 0.531
   Epoch  4: Train=0.458, Val=0.464, Loss=1.1761, Time=154.7s
      ⭐ NEW BEST: 0.464
-------------------------------------------------

In [ ]:
# PointNet Evaluation and Results
print("📊 POINTNET EVALUATION (3D Point Clouds)")
print("="*50)

# Test evaluation
model.eval()
test_correct, test_total = 0, 0
all_preds, all_labels = [], []

with torch.no_grad():
    for points, labels in test_loader:
        points, labels = points.to(DEVICE), labels.to(DEVICE)
        outputs = model(points)
        _, preds = torch.max(outputs, 1)
        
        test_correct += torch.sum(preds == labels).item()
        test_total += labels.size(0)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = test_correct / test_total

print(f"🌳 PointNet Results:")
print(f"   Training time: {training_time:.1f}s")
print(f"   Best val accuracy: {best_val_acc:.3f}")
print(f"   Test accuracy: {test_accuracy:.3f}")
print(f"   Method: Direct 3D Point Cloud Processing")
print(f"   Status: {'✅ SUCCESS' if test_accuracy > 0.7 else '⚠️ ROOM FOR IMPROVEMENT'}")

# Detailed classification report
print(f"\n📋 Classification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))

# Visualize results
plt.figure(figsize=(15, 5))

# Training curves
plt.subplot(1, 3, 1)
epochs = range(1, len(history['train_acc']) + 1)
plt.plot(epochs, history['train_acc'], 'b-', label='Train Acc')
plt.plot(epochs, history['val_acc'], 'r-', label='Val Acc')
plt.axhline(y=test_accuracy, color='g', linestyle='--', label=f'Test Acc ({test_accuracy:.3f})')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('PointNet Training Progress\n(3D Point Clouds)')
plt.legend()
plt.grid(True)

# Loss curves
plt.subplot(1, 3, 2)
plt.plot(epochs, history['train_loss'], 'b-', label='Train Loss')
plt.plot(epochs, history['val_loss'], 'r-', label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves')
plt.legend()
plt.grid(True)

# Confusion matrix
plt.subplot(1, 3, 3)
cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title(f'PointNet Results\n{test_accuracy:.1%} accuracy')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.tight_layout()
plt.show()
#evaluation metrics

print(f"Overall Accuracy: {test_accuracy:.3f}")
print(f"Balanced Accuracy: {accuracy_score(all_labels, all_preds):.3f}")
print(f"F1 Score: {f1_score(all_labels, all_preds, average='weighted'):.3f}")
precision = precision_score(all_labels, all_preds, average=None)
for i, p in enumerate(precision):
    print(f"   Precision for class {class_names[i]}: {p:.3f}")
print(f"Execution Time: {training_time:.3f}s")


#• Number of Parameters (Model Complexity)
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params:,}")


# Save model
model_path = "../../../models/pointnet_3d_model.pth"
torch.save({
    'model_state_dict': model.state_dict(),
    'class_names': class_names,
    'label_encoder': label_encoder,
    'test_accuracy': test_accuracy,
    'history': history,
    'model_config': {
        'num_classes': num_classes,
        'num_points': NUM_POINTS
    }
}, model_path)

print(f"\n💾 PointNet model saved: {model_path}")
print(f"🌳 POINTNET 3D CLASSIFICATION COMPLETE!")
print(f"   Method: Direct 3D point cloud processing")
print(f"   Features: FPS sampling + normalization + PointNet")
print(f"   Final accuracy: {test_accuracy:.1%}")
print(f"   3D deep learning implementation ✅")